In [1]:
# import the needed packages
# from result_fusion import Fusion
from result_fusion import Fusion2
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
from tools.iou_perimg import SegmentationMetrics
import tqdm

In [2]:

# define the folder path and parameters
# train2 is the folder of the generated mask, preciser
# train is the folder of the generated mask, default sam params
mask_folder = '/media/ywh/1/yanweihao/projects/segmentation/segment-anything/outputs/cityscapes/train2'
# the path to the model prediction
# segmentation_root = '/media/ywh/1/yanweihao/projects/uda/DAFormer/work_dirs'
segmentation_folder = '/media/ywh/1/yanweihao/projects/uda/DAFormer/work_dirs/local-exp7/230522_2312_gta2cs_dacs_a999_fdthings_rcs001_cpl_daformer_sepaspp_mitb5_poly10warm_s0_ea659/trainid'
# segmentation_folder = '/media/ywh/1/yanweihao/projects/uda/MIC/seg/work_dirs/local-basic/230509_1455_gtaHR2csHR_mic_hrda_s2_108c1/pred_trainid'
# the path to the original image
image_folder = '/media/ywh/1/yanweihao/dataset/cityscapes_original/gtFine_trainvaltest/leftImg8bit/train_all'
# the path to the ground truth
gt_folder = '/media/ywh/1/yanweihao/dataset/cityscapes_original/gtFine_trainvaltest/gtFine/train_all'

# 
mix_ratio = 0.5
# 
resize_ratio = 0.5
# 
output_folder = 'outputs/cityscapes/debug_tmp' #这是去掉了mask按照名称排序的过程
# 
mask_suffix = '.png'
# 
# segmentation_suffix = '_gtFine_labelTrainIds.png'
# segmentation_suffix = '_leftImg8bittrainID.png'
segmentation_suffix = '_leftImg8bittrainID.png'
#
gt_suffix = '_gtFine_labelTrainIds.png'
# 
segmentation_suffix_noimg=False
# fusion mode = 1
# fusion_mode = 0
fusion_mode = 3
# 
sam_classes = [11, 12, 13, 14, 15, 16, 17, 18]  # 11 classes, 5, 6, 7, 
# 
shrink_num = 2
#
display_size = (400, 800)
# whether to save the mixed result
save_mix_result = True

In [3]:
fusion = Fusion2(mask_folder, segmentation_folder, image_folder, gt_folder, \
                mix_ratio, resize_ratio, output_folder, mask_suffix, \
                segmentation_suffix, segmentation_suffix_noimg, gt_suffix, \
                fusion_mode, sam_classes, shrink_num, display_size)

debug_num = 5 #2975
begin_index = 0 #0
index_range = list(range(begin_index, begin_index + debug_num))
iou_cal = SegmentationMetrics()

bar = tqdm.tqdm(total=debug_num)
for i in index_range:
    image_name = fusion.image_names[i]  # aachen_000000_000019_leftImg8bit
    # get the prediction
    prediction_path = os.path.join(fusion.segmentation_folder, \
        image_name.replace('_leftImg8bit', '') + fusion.segmentation_suffix)
    if fusion.segmentation_suffix_noimg:
        prediction_path = prediction_path.replace('_leftImg8bit', '')
    segmentation = cv2.imread(prediction_path) #[h, w, 3], 3 channels not 1 channel
    # print(np.unique(segmentation))
    segmentation_color = fusion.color_segmentation(segmentation[:,:,0])
    # get the ground truth
    gt_path = os.path.join(fusion.gt_folder, \
        image_name.replace('_leftImg8bit', '') + fusion.gt_suffix)
    gt = cv2.imread(gt_path) #[h, w, 3]
    # print(np.unique(gt))
    gt_color = fusion.color_segmentation(gt[:,:,0])
            
    #get the original image
    original_image = cv2.imread(os.path.join(fusion.image_folder, \
        image_name + fusion.mask_suffix))
    #get the sam segmentation result using the mask
    sam_pred = fusion.get_sam_pred(image_name, segmentation)  # [h,w]
    sam_color = fusion.color_segmentation(sam_pred)  # [h,w,3]
    
    #get the sam mixed color image using the fusion.mix_ratio
    sam_mixed_color = cv2.addWeighted(original_image, \
        fusion.mix_ratio, sam_color, 1 - fusion.mix_ratio, 0)
    if fusion.resize_ratio != 1:
        sam_mixed_color = cv2.resize(sam_mixed_color, \
        (int(sam_mixed_color.shape[1] * fusion.resize_ratio), \
        int(sam_mixed_color.shape[0] * fusion.resize_ratio)), \
        interpolation=cv2.INTER_NEAREST)
    
    if save_mix_result:
        #save the sam mask in trainid and color to the output folder
        cv2.imwrite(os.path.join(fusion.output_folder, 'trainID', image_name + fusion.mask_suffix), sam_pred)
        # cv2.imwrite(os.path.join(fusion.output_folder, 'color', image_name + fusion.mask_suffix), fusion_color)
        cv2.imwrite(os.path.join(fusion.output_folder, 'mixed', image_name + fusion.mask_suffix), sam_mixed_color)
    
        if fusion.fusion_mode == 1:
            fusion_trainid_bg, fusion_color_bg = fusion.fusion_mode_1(segmentation=segmentation, sam_pred=sam_pred)
        elif fusion.fusion_mode == 2:
            fusion_trainid_bg, fusion_color_bg = fusion.fusion_mode_2(segmentation=segmentation, sam_pred=sam_pred)
        elif fusion.fusion_mode == 3:
            fusion_trainid_bg, fusion_color_bg = fusion.fusion_mode_3(segmentation=segmentation, sam_pred=sam_pred)
        elif fusion.fusion_mode == 4:
            fusion_trainid_bg, fusion_color_bg = fusion.fusion_mode_4(segmentation=segmentation, sam_pred=sam_pred)
        else:
            # raise NotImplementedError
            raise NotImplementedError("This fusion mode has not been implemented yet.")
    
        #save the fusion mask in trainid and color to the output folder
        mixed_color_bg = cv2.addWeighted(original_image, fusion.mix_ratio, fusion_color_bg, 1 - fusion.mix_ratio, 0)
        if fusion.resize_ratio != 1:
            mixed_color_bg = cv2.resize(mixed_color_bg, (int(mixed_color_bg.shape[1] * fusion.resize_ratio), int(mixed_color_bg.shape[0] * fusion.resize_ratio)), interpolation=cv2.INTER_NEAREST)
        cv2.imwrite(os.path.join(fusion.output_folder, 'trainID_bg', image_name + fusion.mask_suffix), fusion_trainid_bg)
        # cv2.imwrite(os.path.join(fusion.output_folder, 'color_bg', image_name + fusion.mask_suffix), fusion_color_bg)
        cv2.imwrite(os.path.join(fusion.output_folder, 'mixed_bg', image_name + fusion.mask_suffix), mixed_color_bg)

    #fusion 1, 2, 3, 4
    fusion_trainid_bg_1, fusion_color_bg_1 = \
        fusion.fusion_mode_1(segmentation=segmentation, sam_pred=sam_pred)
    fusion_trainid_bg_2, fusion_color_bg_2 = \
        fusion.fusion_mode_2(segmentation=segmentation, sam_pred=sam_pred)
    fusion_trainid_bg_3, fusion_color_bg_3 = \
        fusion.fusion_mode_3(segmentation=segmentation, sam_pred=sam_pred)
    fusion_trainid_bg_4, fusion_color_bg_4 = \
        fusion.fusion_mode_4(segmentation=segmentation, sam_pred=sam_pred)
    
    miou_1, ious_1 = iou_cal.calculate_miou(fusion_trainid_bg_1, gt[:,:,0])
    miou_2, ious_2 = iou_cal.calculate_miou(fusion_trainid_bg_2, gt[:,:,0])
    miou_3, ious_3 = iou_cal.calculate_miou(fusion_trainid_bg_3, gt[:,:,0])
    miou_4, ious_4 = iou_cal.calculate_miou(fusion_trainid_bg_4, gt[:,:,0])
    
    error_1 = fusion.get_error_image(fusion_trainid_bg_1, gt[:,:,0], fusion_color_bg_1)
    error_2 = fusion.get_error_image(fusion_trainid_bg_2, gt[:,:,0], fusion_color_bg_2)
    error_3 = fusion.get_error_image(fusion_trainid_bg_3, gt[:,:,0], fusion_color_bg_3)
    error_4 = fusion.get_error_image(fusion_trainid_bg_4, gt[:,:,0], fusion_color_bg_4)
    
    # fusion.display_images_horizontally([gt_color, sam_color, segmentation_color, fusion_color_bg_0], '{}_fusion0'.format(image_name.replace('_leftImg8bit', '')), miou_0)
    fusion.display_images_horizontally(
        [original_image, gt_color, sam_color, segmentation_color, \
        fusion_color_bg_1, fusion_color_bg_2, fusion_color_bg_3, fusion_color_bg_4, \
        error_1, error_2, error_3, error_4], \
        '{}'.format(image_name.replace('_leftImg8bit', '')), \
        [(miou_1, ious_1), (miou_2, ious_2),(miou_3, ious_3), (miou_4, ious_4)])
    fusion.save_ious(miou_1, ious_1, miou_2, ious_2, miou_3, ious_3, miou_4, ious_4, \
        '{}'.format(image_name.replace('_leftImg8bit', '')))
    bar.update(1)
bar.close()
 

the path is already exist
the path is already exist
the path is already exist
the path is already exist
the path is already exist
the path is already exist


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:34<00:00,  6.89s/it]
